In [ ]:
# _*_ coding: utf-8 _*_
# @author: Cover WU
# date: 2021-02-13
# in reference of Dr. Liao's code work.

import numpy as np
import pandas as pd
import os
import cv2 as cv
from math import sqrt

from google.colab import drive
drive.mount('/content/Google_Drive')

print("Runtime completed.")

Mounted at /content/Google_Drive
Runtime completed.


In [ ]:
!unzip /content/Google_Drive/MyDrive/seg_results_3.zip

In [ ]:
def identify_fixation_semantics(path, seg_path,filename):
    # define the constant
    FPS = 30
    FRAME_DURATION = 1000 / FPS

    # read the fixation data into dataframe list
    output = path + 'labelled_eventlist.csv'
    readin = path + filename
    fixation_dataframe = pd.read_table(readin)

    def timestamp_from_float_to_integer(column):
        column = np.round(column)
        column = pd.to_numeric(column, downcast='integer')
        column[column == 0] = 1
        return column

    # correct frame number
    fixation_dataframe['StartFrameNo'] = timestamp_from_float_to_integer(fixation_dataframe['Start'] / FRAME_DURATION)
    fixation_dataframe['StopFrameNo'] = timestamp_from_float_to_integer(fixation_dataframe['Stop'] / FRAME_DURATION)

    # compute the position of each fixation
    index_x = (fixation_dataframe['FixationPointX'] > 0) & (fixation_dataframe['FixationPointX'] < 1)
    index_y = (fixation_dataframe['FixationPointX'] > 0) & (fixation_dataframe['FixationPointY'] < 1)
    fixation_dataframe.loc[index_x, 'FrameX'] = np.floor(WIDTH * fixation_dataframe['FixationPointX'])
    fixation_dataframe.loc[index_y, 'FrameY'] = np.floor(HEIGHT * fixation_dataframe['FixationPointY'])
    fixation_dataframe['FrameX'] = pd.to_numeric(fixation_dataframe['FrameX'], downcast='integer')
    fixation_dataframe['FrameY'] = pd.to_numeric(fixation_dataframe['FrameY'], downcast='integer')

    media_list = os.listdir(seg_path)
    fixation_dataframe_labelled = fixation_dataframe[fixation_dataframe['Recording'] == 'p']

    for media in media_list:
        fixation_in_media = fixation_dataframe[fixation_dataframe['Media'] == media]
        for index, row in fixation_in_media.iterrows():
            x, y = row['FrameX'], row['FrameY']
            if (not np.isnan(x)) & (not np.isnan(y)):
                x = np.int(x)
                y = np.int(y)
                start_frame_no = row['StartFrameNo']
                stop_frame_no = row['StopFrameNo']

                img_path = seg_path + '/' + media + '/'
                label_color = get_label_color(x, y, start_frame_no, stop_frame_no, img_path)
                label = get_class_label(label_color)

                fixation_in_media.loc[index, 'Label'] = label
                show = "%s, %s, Fixation %4d, labelled as %s" % (row['Participant'], row['Media'], row['EventIndex'], label)
                print(show)
        fixation_dataframe_labelled = fixation_dataframe_labelled.append(fixation_in_media)

    fixation_dataframe_labelled.to_csv(output, index=False)

def get_class_label(label_color):
    # label_color is a color tuple.
    labels = {
        (128,  64, 128): 'road',
        (244,  35, 232): 'sidewalk',
        ( 70,  70,  70): 'building',
        (102, 102, 156): 'wall',
        (190, 153, 153): 'fence',
        (153, 153, 153): 'pole',
        (250, 170,  30): 'traffic light',
        (220, 220,   0): 'traffic sign',
        (107, 142,  35): 'vegetation',
        (152, 251, 152): 'terrain',
        ( 70, 130, 180): 'sky',
        (220,  20,  60): 'person',
        (255,   0,   0): 'rider',
        (  0,   0, 142): 'car',
        (  0,   0,  70): 'truck',
        (  0,  60, 100): 'bus',
        (  0,  80, 100): 'train',
        (  0,   0, 230): 'motorcycle',
        (119,  11,  32): 'bicycle'
    }
    if label_color in labels.keys():
        return labels[label_color]
    else: return 'unlabelled'


def get_label_color(x, y, start_frame_no, stop_frame_no, img_path):
    '''
    function:
        for a certain fixation corresponding to multiple frames, get its color tuple and return it.

    argument:
        x, y: the location of the fixation center.

    '''
    DISPERSION = 25
    color_dict = {}

    # for each frame (below):
    for i in range(start_frame_no, stop_frame_no + 1):
        imgfile = img_path + 'frame%06d' % i + '.png'
        if not os.path.exists(imgfile):
            break
        img = cv.imread(imgfile)

        # a square buffer
        min_x = 0 if x - DISPERSION < 0 else x - DISPERSION
        max_x = WIDTH - 1 if x + DISPERSION >= WIDTH else x + DISPERSION
        min_y = 0 if y - DISPERSION < 0 else y - DISPERSION
        max_y = HEIGHT - 1 if y + DISPERSION >= HEIGHT else y + DISPERSION

        pixel_color_dict = {}
        # x_i, y_i is a scanning cursor in buffer to capture color vector.
        for x_i in range(min_x, max_x + 1):
            for y_i in range(min_y, max_y + 1):
                if sqrt((x - x_i) ** 2 + (y - y_i) ** 2) > 25:
                    continue
                pixel_color = (img[y_i, x_i][0], img[y_i, x_i][1], img[y_i, x_i][2])
                if pixel_color in pixel_color_dict.keys():
                    pixel_color_dict[pixel_color] += 1
                else:
                    pixel_color_dict[pixel_color] = 1

        if len(pixel_color_dict) > 0:
            major_color = max(pixel_color_dict, key=pixel_color_dict.get)
        else:
            major_color = (0, 0, 0)

        if major_color in color_dict.keys():
            color_dict[major_color] += 1
        else:
            color_dict[major_color] = 1
    if len(color_dict) > 0:
        return max(color_dict, key=color_dict.get)
    else:
        return (0, 0, 0)


In [6]:
if __name__ == '__main__':
    WIDTH, HEIGHT = 1920, 1080

    readinpath = r'/content/Google_Drive/MyDrive/'
    segpath = r'/content/content/cityscapes_seg_results'
    filename = r'UrbanVitality_20210110 Metrics_event_edited.tsv'
    ans = identify_fixation_semantics(readinpath, segpath, filename)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


流式输出内容被截断，只能显示最后 5000 行内容。
201911011116, Night_L, Fixation  110, labelled as car
201911011116, Night_L, Fixation  111, labelled as car
201911011116, Night_L, Fixation  112, labelled as road
201911011116, Night_L, Fixation  113, labelled as fence
201911011116, Night_L, Fixation  114, labelled as car
201911011116, Night_L, Fixation  115, labelled as car
201911011116, Night_L, Fixation  116, labelled as road
201911011116, Night_L, Fixation  117, labelled as car
201911011116, Night_L, Fixation  118, labelled as building
201911011116, Night_L, Fixation  119, labelled as fence
201911011116, Night_L, Fixation  120, labelled as building
201911011116, Night_L, Fixation  121, labelled as building
201911011116, Night_L, Fixation  122, labelled as building
201911011116, Night_L, Fixation  123, labelled as building
201911011116, Night_L, Fixation  124, labelled as building
201911011116, Night_L, Fixation  125, labelled as vegetation
201911011116, Night_L, Fixation  126, labelled as car
201911011116